In [1]:
import json
import sys

In [3]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import jax
import jax.numpy as jnp

import distrax
from distrax import HMM

from dynamax.hidden_markov_model import CategoricalHMM

import optax

import tensorflow_probability as tfp
from tensorflow_probability.substrates import jax as tfpj

import scipy.stats as stats

2025-02-05 08:23:01.270298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738743781.287309   19363 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738743781.292097   19363 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Get a MSA from a set of unaligned sequences by learning the consensus parameters of the profile HMM

In [4]:
test_seq_id = "HBA_HUMAN"

sample_seqs = {
  'HBA_AEGMO': 'MVLSANDKTNVKTVFTKITGHAEDYGAETLERMFITYPPTKTYFPHFDLHHGSAQIKAHGKKVVGALIEAVNHIDDIAGALSKLSDLHAQKLRVDPVNFKLLGQCFLVVVAIHHPSVLTPEVHASLDKFLCAVGNVLTAKYR',
  'HBA_AILFU': 'MVLSPADKTNVKSTWDKLGGHAGEYGGEALERTFASFPTTKTYFPHFDLSPGSAQVKAHGKKVADALTLAVGHLDDLPGALSALSDLHAHKLRVDPVNFKLLSHCLLVTLACHHPAEFTPAVHASLDKFFSAVSTVLTSKYR',
  'HBA_ALCAA': 'MVLSATDKSNVKAAWGKVGGNAPAYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKAHGEKVANALTKAVGHLDDLPGTLSDLSDLHAHKLRVDPVNFKLLSHTLLVTLAAHLPSDFTPAVHASLDKFLANVSTVLTSKYR',
  'HBA_ALLMI': 'MVLSMEDKSNVKAIWGKASGHLEEYGAEALERMFCAYPQTKIYFPHFDMSHNSAQIRAHGKKVFSALHEAVNHIDDLPGALCRLSELHAHSLRVDPVNFKFLAHCVLVVFAIHHPSALSPEIHASLDKFLCAVSAVLTSKYR',
  'HBA_AMMLE': 'MVLSAADKSNVKAAWGKVGGNAGAYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGEKVAAALTKAVGHLDDLPGTLSDLSDLHAHKLRVDPVNFKLLSHSLLVTLACHLPNDFTPAVHASLDKFLANVSTVLTSKYR',
  'HBA_AQUCH': 'MVLSANDKTNVKNVFTKISGHAEDYGAEALERMFTTYPPTKTYFPHFDLHHGSAQIKAHGKKVVGALIEAVNHIDDMAGALSKLSDLHAQKLRVDPVNFKLLGQCFLVVVAIHHPSVLTPEVHASLDKFLCAVGNVLTAKYR',
  'HBA_ATEGE': 'MVLSPADKSNVKAAWGKVGGHAGDYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHHPADFTPAVHASLDKFLASVSTVLTSKYR',
  'HBA_BISBO': 'MVLSAADKGNVKAAWGKVGGHAAEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGAKVAAALTKAVGHLDDLPGALSELSDLHAHKLRVDPVNFKLLSHSLLVTLASHLPNDFTPAVHASLDKFLANVSTVLTSKYR',
  'HBA_BOVIN': 'MVLSAADKGNVKAAWGKVGGHAAEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVNFKLLSHSLLVTLASHLPSDFTPAVHASLDKFLANVSTVLTSKYR',
  'HBA_BRACA': 'VLSAADKTNVKGVFSKIGGHADEYGAETLERMFVAYPQTKTYFPHFDLQHGSAQIKAHGKKVAAALVEAVNHIDDIAGALSKLSDLHAQKLRVDPVNFKFLGHCFLVVVAIHHPSALTPEVHASLDKFLCAVGTVLTAKYR',
  'HBA_CANLF': 'VLSPADKTNIKSTWDKIGGHAGDYGGEALDRTFQSFPTTKTYFPHFDLSPGSAQVKAHGKKVADALTTAVAHLDDLPGALSALSDLHAYKLRVDPVNFKLLSHCLLVTLACHHPTEFTPAVHASLDKFFAAVSTVLTSKYR',
  'HBA_CARAU': 'SLSDKDKAVVKALWAKIGSRADEIGAEALGRMLTVYPQTKTYFSHWSDLSPGSGPVKKHGKTIMGAVGDAVSKIDDLVGALSALSELHAFKLRIDPANFKILAHNVIVVIGMLFPGDFTPEVHMSVDKFFQNLALALSEKYR',
  'HBA_CAVPO': 'MVLSAADKNNVKTTWDKIGGHAAEYVAEGLTRMFTSFPTTKTYFHHIDVSPGSGDIKAHGKKVADALTTAVGHLDDLPTALSTLSDVHAHKLRVDPVNFKFLNHCLLVTLAAHLGADFTPSIHASLDKFFASVSTVLTSKYR',
  'HBA_CHICK': 'MVLSAADKNNVKGIFTKIAGHAEEYGAETLERMFTTYPPTKTYFPHFDLSHGSAQIKGHGKKVVAALIEAANHIDDIAGTLSKLSDLHAHKLRVDPVNFKLLGQCFLVVVAIHHPAALTPEVHASLDKFLCAVGTVLTAKYR',
  'HBA_HORSE': 'MVLSAADKTNVKAAWSKVGGHAGEYGAEALERMFLGFPTTKTYFPHFDLSHGSAQVKAHGKKVGDALTLAVGHLDDLPGALSNLSDLHAHKLRVDPVNFKLLSHCLLSTLAVHLPNDFTPAVHASLDKFLSSVSTVLTSKYR',
  'HBA_HUMAN': 'MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR'
}
unaligned_seqs = [v for k,v in sample_seqs.items() if k != test_seq_id]
test_seq = sample_seqs[test_seq_id]
print('MSA seqs: ', len(unaligned_seqs)) 
print('test seq length:', len(test_seq))

MSA seqs:  15
test seq length: 142


In [6]:
# all amino acids + gap
vocab = 'ARNDCQEGHILKMFPSTWYV-'
n_vocab = len(vocab)
n_vocab

21

In [7]:
states = 'MID'  # Match, Insert, Delete
n_states = len(states)
n_states

3

#### Gradient descent

In [8]:
hmm = CategoricalHMM(num_states=n_states, emission_dim=1, num_classes=n_vocab)

# Initialize the model
key = jax.random.PRNGKey(0)
params, props = hmm.initialize(key)
params, props

(ParamsCategoricalHMM(initial=ParamsStandardHMMInitialState(probs=Array([0.21115594, 0.6283457 , 0.16049828], dtype=float32)), transitions=ParamsStandardHMMTransitions(transition_matrix=Array([[0.44149017, 0.30103576, 0.257474  ],
        [0.01715945, 0.32746994, 0.6553706 ],
        [0.12272572, 0.10252184, 0.77475244]], dtype=float32)), emissions=ParamsCategoricalHMMEmissions(probs=Array([[[0.00886245, 0.02676938, 0.0558548 , 0.07991356, 0.01317011,
          0.00257393, 0.02607093, 0.01364831, 0.05251984, 0.06331246,
          0.16337217, 0.01088525, 0.05258397, 0.05338747, 0.02124663,
          0.04021688, 0.03417913, 0.02328008, 0.14380595, 0.09843415,
          0.01591237]],
 
        [[0.03102212, 0.03003852, 0.01944453, 0.03899414, 0.03831979,
          0.01327157, 0.08538215, 0.05602564, 0.01902615, 0.06713416,
          0.00358675, 0.23546287, 0.01309458, 0.08970489, 0.07291738,
          0.06335647, 0.01522192, 0.01848631, 0.00524298, 0.00998701,
          0.07428014]],
 
  